# Use Requests Library with Bria in Azure AI and Azure ML for Text to Image requests

Use requests library to consume Bria AI deployments in Azure AI and Azure ML.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription

* Make sure you have access to Azure AI Studio

* Create a project and resource group

* Select Bria AI models from Model catalog. This example assumes you are Bria 2.3.

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation here for model deployment and inference.

To complete this tutorial, you will need to:

* Install requests:
    
    pip install requests

* Install pillow:
    
    pip install pillow

* Install matplotlib
    
    pip install matplotlib

## Example
The following is an example about how to use requests library with a Bria AI Text to Image model deployed in Azure AI and Azure ML:



In [1]:
import base64
import requests
from typing import Any
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


In [6]:

class ImageGeneratorClient:
    def __init__(self, base_url, key):
        """Initialize the client with the endpoint URL and Bearer token."""
        self.route = base_url + "/images/generations"
        self.headers = {
            "Authorization": f"Bearer {key}",
            "Content-Type": "application/json",
            "accept": "application/json",
            "extra-parameters": "pass-through"
        }

    def generate_image(self, prompt, size, seed=None, negative_prompt=None, image_format=None, num_inference_steps=None, text_guidance=None):
        """Send a request to the endpoint to generate an image and save the result to a file."""
        payload = {"prompt": prompt, "size": size}
        self._add_paramter_if_provided(payload, "seed", seed)
        self._add_paramter_if_provided(payload, "negative_prompt", negative_prompt)
        self._add_paramter_if_provided(payload, "image_format", image_format)
        self._add_paramter_if_provided(payload, "num_inference_steps", num_inference_steps)
        self._add_paramter_if_provided(payload, "text_guidance", text_guidance)

        response = requests.post(self.route, json=payload, headers=self.headers)
        
        if response.status_code == 200:
            response_data = response.json()
            generated_image_base64 = response_data.get("data")[0].get("b64_json")
            if generated_image_base64:
                self._display_base64_image(generated_image_base64)
            else:
                print("No generated image found in the response.")
        else:
            print(f"Error: {response.status_code}, {response.text}")

    @staticmethod
    def _display_base64_image(base64_string):
        """Decode a base64 string and display it as an image."""
        image_data = base64.b64decode(base64_string)
        image = Image.open(BytesIO(image_data))
        plt.imshow(image)
        plt.axis('off')
        plt.show()
    
    @staticmethod
    def _add_paramter_if_provided(payload: dict, parameter: str, value: Any):
        if value is not None:
            payload[parameter] = value


You will need to have a Endpoint url and Authentication Key associated with that endpoint. This can be acquired from previous steps. Configure the client as follows:

base_url: Use the endpoint URL from your deployment.
api_key: Use your API key.

In [7]:
client = ImageGeneratorClient(
    base_url="",
    key=""
)


Use the client to create image generation requests:

In [ ]:
# Mandatory parameters

prompt = "A tree"
size = "1024x1024"

#Optional parameters
seed = 1
negative_prompt = "birds"
image_format = "jpeg"
num_inference_steps = 10
text_guidance = 2.0

client.generate_image(prompt=prompt,
                      size=size,
                      seed=seed,
                      negative_prompt=negative_prompt,
                      image_format=image_format,
                      num_inference_steps=num_inference_steps,
                      text_guidance=text_guidance)